In [7]:
import importlib

import pandas as pd
import numpy as np 
import csv
import math
import credentials; importlib.reload(credentials)
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot 
import queries
import measurement as mt 
from datetime import date
from dateutil.relativedelta import relativedelta
from datetime import datetime
date_format = "%Y%m%d"
from IPython.display import Markdown
import os

#import widgets 
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

#import plotly 
import plotly as py 
import plotly.graph_objs as go
import ipywidgets as widgets
import numpy as np
from scipy import special

py.offline.init_notebook_mode(connected=True)

%reload_ext autoreload
%autoreload 2

%reload_ext sql 

In [8]:
savepath = '/Users/eliasayoub/Documents/CLCM/Measurement Dashboard'

In [176]:
#wraps around a sql query that is incoming from another file
def Query(q):
    result = %sql $q
    return result.DataFrame()

# #removes duplicates
def remove_dup(df):
    if len(df)>2:
        cleaned = df.drop_duplicates(subset='contacted', keep='first')
        return cleaned
    else:
        return df


def pre_acv_atv_atf(start_dt, pre_period_days):
    from datetime import date
    from dateutil.relativedelta import relativedelta
    from datetime import datetime
    date_format = "%Y%m%d"
    import queries
    
    a = datetime.strptime(str(start_dt), date_format)

    norm_start = a - relativedelta(days=pre_period_days) #range of pre-period 
    norm_end = a - relativedelta(days=1)
    pre_start_dt = norm_start.strftime("%Y%m%d")
    pre_end_dt = norm_end.strftime("%Y%m%d") ##good so far!

    def Query(q):
        result = %sql $q
        return result.DataFrame()
    
    hist = Query(queries.acv_check(campaign_name, pre_start_dt, pre_end_dt, id_seg))
    
    def cx_check(metrics_table, hist):
        cx = pd.DataFrame({
        'ACV' : [metrics_table.loc['ACV','Target'], metrics_table.loc['ACV','Control']],
        'ATV' : [metrics_table.loc['ATV', 'Target'], metrics_table.loc['ATV','Control']],
        'ATF' : [metrics_table.loc['ATF', 'Target'], metrics_table.loc['ATF','Control']],
        'Group' : ['Target', 'Control']
        })
        cx = cx.set_index('Group')


        hist = hist.groupby('control').sum()

        hist1 = pd.DataFrame()
        hist1['ACV Pre'] = hist.loc['N', 'sales']/hist.loc['N', 'universe'], hist.loc['Y', 'sales']/hist.loc['Y', 'universe']
        hist1['ATV Pre'] = hist.loc['N', 'sales']/hist.loc['N', 'trans'], hist.loc['Y', 'sales']/hist.loc['Y', 'trans']
        hist1['ATF Pre'] = hist.loc['N', 'trans']/hist.loc['N', 'universe'], hist.loc['Y', 'trans']/hist.loc['Y', 'universe']
        hist1['Group'] = ['Target', 'Control']
        hist1 = hist1.set_index('Group')


        hist1 = hist1.round(2)


        both = pd.concat([cx,hist1], axis=1)
        both = both[['ACV', 'ACV Pre', 'ATV', 'ATV Pre', 'ATF', 'ATF Pre']]

        return both
    
    check_metrics = cx_check(metrics_table, hist)
    return check_metrics


def intWithCommas(x):
    if type(x) not in [type(0), type(int)]:
        raise TypeError("Parameter must be an integer.")
    if x < 0:
        return '-' + intWithCommas(-x)
    result = ''
    while x >= 1000:
        x, r = divmod(x, 1000)
        result = ",%03d%s" % (r, result)
    return "%d%s" % (x, result)
    
    
def normalize_q(norm_start, norm_end, campaign_name, id_seg, id_treatment):
    
    pre_period_query = """
    with contacts as( 
    select distinct campaign_name, treatment_name, segment_name, 
    case when control_group_flag='Y' 
    or segment_name like '%CTRL%' 
    or segment_name like '%_CT%' 
    or segment_name like '%CONTRL%'
    or segment_name like '%CONTROL%'
    or segment_name like '%_CT_%'
    or segment_name like '%control%'
    or segment_name like '%Control%'
    or campaign_name like '%CT%'
    
    then 'Y' else 'N' end as control,
    contact_key, campaign_start_dt, campaign_end_dt

    from rawdata.b_camp_offer_rk a join rawdata.b_camp_contact_rk b using (camp_offer_key)

    where campaign_name like '{campaign_name}'
    and {id_seg}
    and {id_treatment}
    ),

    segment as(
    select distinct campaign_name, treatment_name, segment_name, 
    case when control_group_flag='Y' 
    or segment_name like '%CTRL%' 
    or segment_name like '%_CT%' 
    or segment_name like '%CONTRL%'
    or segment_name like '%CONTROL%'
    or segment_name like '%_CT_%'
    or segment_name like '%control%'
    or segment_name like '%Control%'
    or campaign_name like '%CT%'
      
    then 'Y' else 'N' end as control,
    count(distinct contact_key) as contacted 

    from rawdata.b_camp_offer_rk join rawdata.b_camp_contact_rk using (camp_offer_key)

    where campaign_name like '{campaign_name}'
    and {id_seg}
    and {id_treatment}

    group by 1,2,3,4
    ),
 
    trans as (
    select * from rawdata.b_transaction_rk
    where transaction_dt_key >= {norm_start} and transaction_dt_key <= {norm_end}
    and contact_key > 0
    and transaction_type_name = 'Item'
    )

    select campaign_name, treatment_name, segment_name, control, contacted,
    count(distinct b.contact_key)::int as active, 
    sum(item_amt)::float as act_sales,
    sum(item_spread_discount_amt)::float as act_discount,
    count(distinct b.order_num)::int as act_trans,
    sum(item_quantity_val)::int as act_items

    from contacts a
    left join trans b on a.contact_key = b.contact_key
    join segment using (campaign_name, treatment_name, segment_name, control)


    group by 1,2,3,4,5 """.format(campaign_name=campaign_name, id_seg=id_seg, norm_start=norm_start, norm_end=norm_end, id_treatment=id_treatment)
    
    result = %sql $pre_period_query
    pre_query = result.DataFrame()
    return pre_query

    
    






#save metrics table and notebook to directory named after the campaign name
def save(metrics_table, savepath, title): 
    import os
    new_path = savepath + '/PCA-' + title
    
    if not os.path.exists(new_path):
        os.makedirs(new_path)
      
    file_name = '/PCA-' + title + '.ipynb'
    os.rename(savepath+'/copy_of_dashboard.ipynb', new_path+file_name)
    metrics_table.to_csv(new_path + '/PCA-' + title + '.csv', index=True)
    return "saved to " + new_path


#appends results to a summary tracker - to be used in this notebook
def append_results(tracker_path, metrics_table, channel, segment, campaign_type):
    #default inputs:
    camp_count = 1
    program = "Managed Campaigns" 
    date = start_dt 

    sales = metrics_table.loc['Total Sales','Target']
    incremental = metrics_table.loc['Incremental Sales', 'Target']
    touchpoints = metrics_table.loc['Universe', 'Target']
    resp_rate = float(metrics_table.loc['Active Members', 'Target'])/float(metrics_table.loc['Universe', 'Target'])
    
    tracker = pd.read_csv(tracker_path)
    index =['Campaign Name', 'Campaign Count', 'Program', 'BU', 'Channel', 'Segment', 'Start Date', 'End Date', 'Rubikloud Value Added', 'Sales', 'RK Incremental', 'Touchpoints', 'Response Rate']
    row = pd.Series([title, camp_count, program, BU, channel, segment, date, campaign_type, sales, incremental, touchpoints, resp_rate]
                , index=index)
    updated_tracker = tracker.append(row, ignore_index=True)
    return updated_tracker

#new, probably still doesnt work: 
def append_results(tracker_path, metrics_table, channel, segment, models_used):
    #default inputs:
    camp_count = 1
    program = "Managed Campaigns" 
    date = start_dt 

    sales = metrics_table.loc['Total Sales','Target']
    incremental = metrics_table.loc['Incremental Sales', 'Target']
    touchpoints = metrics_table.loc['Universe', 'Target']
    resp_rate = float(metrics_table.loc['Active Members', 'Target'])/float(metrics_table.loc['Universe', 'Target'])
    
    tracker = pd.read_csv(tracker_path)
    index =['Campaign Name', 'Campaign Count', 'Program', 'BU', 'Channel', 'Segment', 'Start Date', 'End Date', 'Rubikloud Value Added', 'Sales', 'RK Incremental', 'Touchpoints', 'Response Rate']
    row = pd.Series([title, camp_count, program, BU, channel, segment, date, models_used, sales, incremental, touchpoints, resp_rate]
                , index=index)
    updated_tracker = tracker.append(row, ignore_index=True)
    return updated_tracker



# def compare_uplift(tracker_path, channel, segment):
#     table = pd.read_csv(tracker_path, error_bad_lines=False, header=0)
#     similar_campaigns = table[table.Channel.str.contains(channel) == True 
#                                          & table.Segment.str.contains(segment) == True]
    
#     avg_uplift = similar_campaigns.groupby('')
    
#save the dataframe tracker to a csv tracker
def to_tracker(appended_df, savepath):
    dt = date.today() 
    dt = dt.strftime("%Y%m%d")
    name = appended_df.iloc[-1,0]
    appended_df.to_csv(savepath+"Results_Tracker_"+dt+".csv", index=False)
    return "Campaign "+name + " has been appended"

#working:

    


### Input Measurement Details Here:

In [177]:
##ANALYST INPUTS##
BU = 'MAT'

campaign_name = '%MAT_1117_EC_Cybermonday%' #An approximate match; use % 
start_dt = 20171127 #YYYYMMDD - ensure dates are provided by the client as PM dates can be inacurate
end_dt = 20171130
segments = ['%'] #Alternative identifier for a campaigN
treatments = ['%'] #Alternative identifier for a campaign
top_channel = 'Email'  #Email, SMS, DM
seg_start = '2017-11-26' #YYYY-MM-DD

#Optional#
#If you use the custom query, you don't need the above inputs except for BU 
custom_query = """

with contacts as( 
select distinct campaign_name, segment_name, contact_key, campaign_start_dt, campaign_end_dt, treatment_name

from rawdata.b_camp_offer_rk a join rawdata.b_camp_contact_rk b using (camp_offer_key)

where campaign_name like '%1711MRKE W48 LOREAL (aswOP8336)%'
--and segment_name like '{id_seg}%'
--and treatment_name like '{id_treatment}%'
),

segment as(
select distinct campaign_name,segment_name, treatment_name,



count(distinct contact_key) as contacted 

from rawdata.b_camp_offer_rk join rawdata.b_camp_contact_rk using (camp_offer_key)

where campaign_name like '%1711MRKE W48 LOREAL (aswOP8336)%'
--and segment_name like '{id_seg}%'
--and treatment_name like '{id_treatment}%'

group by 1,2,3
),

trans as (
select * from rawdata.b_transaction_rk
where transaction_dt_key >= 20171127 and transaction_dt_key <= 20171127 --YYYMMDD
and contact_key > 0
and transaction_type_name = 'Item'
)


select campaign_name, segment_name, a.treatment_name, 
case when a.treatment_name = '359371207-aswOFRDummyOfferForExportMFR-359371207' then 'Y' else 'N' end as control,
contacted,
count(distinct b.contact_key)::int as active, 
sum(item_amt)::float as act_sales,
sum(item_spread_discount_amt)::float as act_discount,
count(distinct b.order_num)::int as act_trans,
sum(item_quantity_val)::int as act_items

from contacts a

left join trans b on a.contact_key = b.contact_key
join rawdata.b_product_rk using (product_key) 
join segment using (campaign_name, segment_name)

--where treatment_name not like '%Dummy%'

--where product_sku in ('100257559', '100375256', '100947111', '100004208', '100899287', '100593376')

where brand_name like '%LANCOME%'
OR brand_name like '%YSL%' 
OR BRAND_NAME LIKE '%ARMANI%' 
OR BRAND_NAME LIKE '%DIESEL%' 
OR BRAND_NAME LIKE '%CACHAREL%' 
--OR BRAND_NAME LIKE '%YVES%SAINT%LAURENT%'
or brand_name in ('YVES SAINT LAURENT', 'YSL', 'Yves Saint Laurent', 'YSL ST LAURENT', 'YVES SAINT LAURENT-MAKE-UP', 'YVES SAINT LAURENT - PARFUMERY', 'YVES SAINT LAURENT L', 'YVES SAINT LAURENT L')


group by 1,2,3,4,5;
"""


In [178]:
#Do not change: 
id_seg = " or ".join(["segment_name like '{seg_name}'".format(seg_name=seg_name) for seg_name in segments])
id_treatment = " or ".join(["treatment_name like '{id_name}'".format(id_name=id_name) for id_name in treatments])
#campaign_spec = " or ".join(["campaign_name like '{cname}'".format(cname=cname) for cname in campaign_list])

conn = credentials.login['{}'.format(BU)]
%reload_ext sql
%sql $conn

'Connected: eayoub@mndv'

### Run query:

In [180]:
# Query to pull campaign info 
raw = Query(queries.pca_data(campaign_name, start_dt, end_dt, id_treatment, id_seg))
#OR:
# raw = Query(custom_query)


raw = remove_dup(raw)
raw

3 rows affected.


,campaign_name,treatment_name,segment_name,control,contacted,active,act_sales,act_discount,act_trans,act_items
0,MAT_1117_EC_Cybermonday (aswOP8303),aswDM99957-aswOFRDummyOfferForExportMAT-aswDM9...,aswDM99957 - Copy of TARGET CYBER MONDAY,Y,730,38,3910.03,535.12,41,126
1,MAT_1117_EC_Cybermonday (aswOP8303),aswDM99957-aswOFRDummyOfferForExportMAT-aswDM9...,aswDM99957 - Copy of TARGET CYBER MONDAY,N,6572,315,28791.49,4316.23,377,1011
2,MAT_1117_EC_Cybermonday (aswOP8303),aswDM99956-aswOFRDummyOfferForExportMAT-aswDM9...,aswDM99956 - Copy of CONTROL CYBER MONDAY,Y,68406,1431,117788.08,14048.84,1587,3920


In [13]:
#use if garbage rows in query

# raw_clean = raw.drop([2])
# raw_clean = raw_clean.reset_index()
# del raw_clean['index']
# raw_clean

In [14]:
#####summary 
#raw_clean.groupby('treatment_name').sum()

In [15]:
######check unique treatment names
#raw_clean.loc[:,'treatment_name'].unique().tolist()

In [16]:
######use if you need to explicitly label a row as control
# raw_clean.loc[0,'control'] = 'Y'
# raw_clean

### Measurement

In [127]:
top_campaign_name = raw.loc[0,'campaign_name']
Markdown("# Campaign Results for: %s" % title)
# top_campaign_name

# Campaign Results for: MAT_1117_EC_Cybermonday (aswOP8303)

In [18]:
import measurement as mt
metrics_table = mt.measurement2(raw)
metrics_table

,Units,Target,Control
Metric,,,
Universe,Members,6572,69136
Active Members,Members,315,1469
Active Rate,%,5.0%,2.0%
Total Sales,Euro,28791,121698
Total Transactions,Number,377,1628
Total Discount,Euro,4316,14584
ATV,Euro/Trans,76,75
ATF,Trans/Member,1.20,1.11
ACV,Euro/Member,91,83


In [123]:
target_size = int(metrics_table.loc['Universe', 'Target'])
ctrl_size = int(metrics_table.loc['Universe', 'Control'])

ctrl_pct = ctrl_size/(ctrl_size+target_size)
ctrl_pct = "%.2f" % round(ctrl_pct*100,2) + "%"

target_size_formatted=intWithCommas(target_size)

top_target_population = target_size
top_target_percent = ctrl_pct
# "{:,}".format(target_size)
#target_size

Markdown("### Target Group Size: %(tsize)s"% {"tsize": target_size_formatted} )

### Target Group Size: 6,572

In [124]:
target_size

6572

In [20]:
Markdown("### Target Group Size: %(csize)s"% {"csize": ctrl_pct} )

### Target Group Size: 91.32%

In [21]:
# sales_t = float(metrics_table.loc['Total Sales', 'Target'])
# acv_c = float(metrics_table.loc['ACV', 'Control'])
# universe_t = float(metrics_table.loc['Universe', 'Target'])

In [112]:
relativedelta(months=6)

a = datetime.strptime(str(start_dt), date_format)
b = datetime.strptime(str(end_dt), date_format)
delta = b - a

top_duration = Markdown("### Campaign length: %(days)d days" % {"days": delta.days })
top_duration

UsageError: Line magic function `%(days)s` not found.


In [111]:
top_duration

### Campaign length: 3 days

In [109]:
delta

datetime.timedelta(3)

In [114]:
total_sales = int(metrics_table.loc['Total Sales','Target'])
total_sales = intWithCommas(total_sales)
top_total_sales = total_sales

Markdown('## Total Sales: €{}'.format(total_sales))


## Total Sales: €28,791

In [121]:
#Print incremental sales: 
inc = int(metrics_table.loc['Incremental Sales','Target'])
top_incremental_dollars = intWithCommas(inc)

active = metrics_table.loc['Active Rate','Target']
top_incremental_percent = metrics_table.loc['Uplift', 'Target']
Markdown("## Incremental Sales: €{}".format(inc))

## Incremental Sales: €17223

In [120]:
#Print uplift:
Markdown("### Total Uplift from Campaign: %s" % uplift)

### Total Uplift from Campaign: 148.88%

In [26]:
sales_by_day = Query(queries.camp_transactions(campaign_name, start_dt, end_dt))

8 rows affected.


In [56]:
sales_by_day_format = sales_by_day.loc[:,'sales']
sales_by_day_format = sales_by_day_format.reset_index()
sales_by_day_format['label_0'] = 'Day'
sales_by_day_format.columns = ['label_1', 'value', 'label_0']
sales_by_day_format['label_1'] = sales_by_day_format['label_1'] + 1
sales_by_day_format.value = sales_by_day_format.value.map(int)
sales_by_day_format.value = sales_by_day_format.value.round()
sales_by_day_format["label"] = sales_by_day_format["label_0"].map(str) + " " + sales_by_day_format["label_1"].map(str)
sales_by_day_format = sales_by_day_format.drop(['label_1', 'label_0'], axis = 1)
sales_by_day_format = sales_by_day_format[['label', 'value']]

dS = sales_by_day_format

In [57]:
# layout_daily_sales = go.Layout(
#     title='Sales by Day',
#     yaxis=dict(
#         title='Sales (Euros)'
#     ),
#     xaxis=dict(
#         title='Day'
#     )
# )


# trace2=go.Bar(
#     x=max(sales_by_day.sales),
#     y=sales_by_day.sales,
#     marker=dict(
#                 color='rgb(0, 121, 193)'
#     )
# )


# fig = go.Figure(data=[trace2], layout=layout_daily_sales)
# py.offline.iplot(fig)

## Viz: Sales by RFM

In [84]:
import queries

In [85]:
sS = Query(queries.sales_by_rfm(campaign_name, start_dt, end_dt, seg_start, id_seg, id_treatment))

6 rows affected.


In [88]:
sS = sS.loc[:,['seg_num', 'act_sales']]
sS.columns = ['label', 'value']
sS

,label,value
0,VIP,15002.02
1,LOYAL,4644.19
2,NEW,250.03
3,ONE-OFFS,1097.49
4,LAPSED,465.65
5,INFREQUENT,4163.79


In [106]:
# sales_rfm = Query(queries.sales_by_rfm(campaign_name, start_dt, end_dt))
# layout_rfm = go.Layout(
#     title='Sales by RFM',
#     xaxis=dict(
#         title='Sales by RFM'
# ))


# trace_rfm=go.Bar(
#     x=sales_rfm.sales,
#     y=sales_rfm.seg_num,
#     name = 'Sales by RFM',
#     orientation = 'h',
#     marker=dict(color='rgb(0, 121, 193)')
   
# )


# fig_rfm = go.Figure(data=[trace_rfm], layout=layout_rfm)
# py.offline.iplot(fig_rfm)

## Deep Dives

In [91]:
####ANALYST INPUTS#####
pre_period_days = 90

#### ACV/ATV/ATF Pre-Period Checks

In [92]:
pre_metrics = pre_acv_atv_atf(start_dt, pre_period_days)

3 rows affected.


In [93]:
pre_metrics

,ACV,ACV Pre,ATV,ATV Pre,ATF,ATF Pre
Group,,,,,,
Target,91,184.14,76,88.37,1.20,2.08
Control,83,117.55,75,79.03,1.11,1.49


In [94]:
pACV = pd.DataFrame(pre_metrics.loc[:, 'ACV Pre'])
pACV = pACV.transpose()
pACV = pACV.reset_index()
pACV = pACV.drop(['index'], axis =1)
pACV

Group,Target,Control
0,184.14,117.55


In [95]:
pATV = pd.DataFrame(pre_metrics.loc[:, 'ATV Pre'])
pATV = pATV.transpose()
pATV = pATV.reset_index()
pATV = pATV.drop(['index'], axis =1)
pATV

Group,Target,Control
0,88.37,79.03


In [96]:
pATF = pd.DataFrame(pre_metrics.loc[:, 'ATF Pre'])
pATF = pATF.transpose()
pATF = pATF.reset_index()
pATF = pATF.drop(['index'], axis =1)
pATF

Group,Target,Control
0,2.08,1.49


In [134]:
top_start_date = start_dt
top_end_date = end_dt

In [156]:
norm_end = '20171126'
norm_start = '20170827'

In [181]:
import measurement
pre_period_raw = normalize_q(norm_start, norm_end, campaign_name, id_seg, id_treatment)
pre_period_raw
pre_metrics_table = mt.measurement2(pre_period_raw)



spc_t = int(metrics_table.loc['Universe', 'Target'])/int(metrics_table.loc['Total Sales', 'Target'])
spc_c = int(metrics_table.loc['Universe', 'Control'])/int(metrics_table.loc['Total Sales', 'Control'])
spc_t_pre = int(pre_metrics_table.loc['Universe', 'Target'])/int(pre_metrics_table.loc['Total Sales', 'Target'])
spc_c_pre = int(pre_metrics_table.loc['Universe', 'Control'])/int(pre_metrics_table.loc['Total Sales', 'Control'])

normalized_inc_sales = (spc_t - spc_c*(spc_t_pre/spc_c_pre))*int(metrics_table.loc['Universe', 'Target'])
normalized_uplift = normalized_inc_sales/(int(metrics_table.loc['Total Sales', 'Target'])- normalized_inc_sales)

metrics_table1 = metrics_table.copy()
metrics_table1.loc['Incremental Sales', 'Target'] = "%.2d" % round(normalized_inc_sales,0)
metrics_table1.loc['Uplift', 'Target'] = "%.2f" % round(normalized_uplift*100,2) + "%" 

                
metrics_table1

3 rows affected.


,Units,Target,Control
Metric,,,
Universe,Members,6572,69136
Active Members,Members,315,1469
Active Rate,%,5.0%,2.0%
Total Sales,Euro,28791,121698
Total Transactions,Number,377,1628
Total Discount,Euro,4316,14584
ATV,Euro/Trans,76,75
ATF,Trans/Member,1.20,1.11
ACV,Euro/Member,91,83


In [182]:
campaignData=[{"metaType":"Topline","metaKey":"Top Campaign Name","data":top_campaign_name,"dataTarget":"","dataControl":""},
{"metaType":"Topline","metaKey":"Top Start Date","data":top_start_date,"dataTarget":"","dataControl":""},
{"metaType":"Topline","metaKey":"Top End Date","data":top_end_date,"dataTarget":"","dataControl":""},
{"metaType":"Topline","metaKey":"Top Duration ","data":"3 Days","dataTarget":"","dataControl":""},
{"metaType":"Topline","metaKey":"Top Channel","data":top_channel,"dataTarget":"","dataControl":""},
{"metaType":"Topline","metaKey":"Top Incremental Dollars","data":top_incremental_dollars,"dataTarget":"","dataControl":""},
{"metaType":"Topline","metaKey":"Top Incremental Percent","data":top_incremental_percent,"dataTarget":"","dataControl":""},
{"metaType":"Topline","metaKey":"Top Total Sales","data":top_total_sales,"dataTarget":"","dataControl":""},
{"metaType":"Topline","metaKey":"Top Target Population","data":top_target_population,"dataTarget":"","dataControl":""},
{"metaType":"Topline","metaKey":"Top Target Percent","data":top_target_percent,"dataTarget":"","dataControl":""},
{"metaType":"Summary","metaKey":"Universe","data":"Members","dataTarget":metrics_table.loc['Universe', 'Target'],"dataControl":metrics_table.loc['Universe', 'Control']},
{"metaType":"Summary","metaKey":"Active Members","data":"Members","dataTarget":metrics_table.loc['Active Members', 'Target'],"dataControl":metrics_table.loc['Active Members', 'Control']},
{"metaType":"Summary","metaKey":"Active Rate","data":"%","dataTarget":metrics_table.loc['Active Rate', 'Target'],"dataControl":metrics_table.loc['Active Rate', 'Control']},
{"metaType":"Summary","metaKey":"Total Sales","data":"Euro","dataTarget":metrics_table.loc['Total Sales', 'Target'],"dataControl":metrics_table.loc['Total Sales', 'Control']},
{"metaType":"Summary","metaKey":"Total Transactions","data":"Number","dataTarget":metrics_table.loc['Total Transactions', 'Target'],"dataControl":metrics_table.loc['Total Transactions', 'Control']},
{"metaType":"Summary","metaKey":"Total Discount","data":"Euro","dataTarget":metrics_table.loc['Total Discount', 'Target'],"dataControl":metrics_table.loc['Total Discount', 'Control']},
{"metaType":"Summary","metaKey":"ATV","data":"Euro/Trans","dataTarget":metrics_table.loc['ATV', 'Target'],"dataControl":metrics_table.loc['ATV', 'Control']},
{"metaType":"Summary","metaKey":"ATF","data":"Trans/Member","dataTarget":metrics_table.loc['ATF', 'Target'],"dataControl":metrics_table.loc['ATF', 'Control']},
{"metaType":"Summary","metaKey":"ACV","data":"Euro/Member","dataTarget":metrics_table.loc['ACV', 'Target'],"dataControl":metrics_table.loc['ACV', 'Control']},
{"metaType":"Summary","metaKey":"Incremental Sales","data":"Euro","dataTarget":"","dataControl":metrics_table.loc['Incremental Sales', 'Target']},
{"metaType":"Summary","metaKey":"Uplift","data":"%","dataTarget":"","dataControl":metrics_table.loc['Uplift', 'Target']},
{"metaType":"Deep Analysis","metaKey":"Universe","data":"Members","dataTarget":metrics_table1.loc['Universe', 'Target'],"dataControl":metrics_table1.loc['Universe', 'Control']},
{"metaType":"Deep Analysis","metaKey":"Active Members","data":"Members","dataTarget":metrics_table1.loc['Active Members', 'Target'],"dataControl":metrics_table1.loc['Active Members', 'Control']},
{"metaType":"Deep Analysis","metaKey":"Active Rate","data":"%","dataTarget":metrics_table1.loc['Active Rate', 'Target'],"dataControl":metrics_table1.loc['Active Rate', 'Control']},
{"metaType":"Deep Analysis","metaKey":"Total Sales","data":"Euro","dataTarget":metrics_table1.loc['Total Sales', 'Target'],"dataControl":metrics_table1.loc['Total Sales', 'Control']},
{"metaType":"Deep Analysis","metaKey":"Total Transactions","data":"Number","dataTarget":metrics_table1.loc['Total Transactions', 'Target'],"dataControl":metrics_table1.loc['Total Transactions', 'Control']},
{"metaType":"Deep Analysis","metaKey":"Total Discount","data":"Euro","dataTarget":metrics_table1.loc['Total Discount', 'Target'],"dataControl":metrics_table1.loc['Total Discount', 'Control']},
{"metaType":"Deep Analysis","metaKey":"ATV","data":"Euro/Trans","dataTarget":metrics_table1.loc['ATV', 'Target'],"dataControl":metrics_table1.loc['ATV', 'Control']},
{"metaType":"Deep Analysis","metaKey":"ATF","data":"Trans/Member","dataTarget":metrics_table1.loc['ATF', 'Target'],"dataControl":metrics_table1.loc['ATF', 'Control']},
{"metaType":"Deep Analysis","metaKey":"ACV","data":"Euro/Member","dataTarget":metrics_table1.loc['ACV', 'Target'],"dataControl":metrics_table1.loc['ACV', 'Control']},
{"metaType":"Deep Analysis","metaKey":"Incremental Sales","data":"Euro","dataTarget":"","dataControl":metrics_table1.loc['Incremental Sales', 'Target']},
{"metaType":"Deep Analysis","metaKey":"Uplift","data":"%","dataTarget":"","dataControl":metrics_table1.loc['Uplift', 'Target']}]

In [183]:
cD = pd.DataFrame(campaignData)
cD

,data,dataControl,dataTarget,metaKey,metaType
0,MAT_1117_EC_Cybermonday (aswOP8303),,,Top Campaign Name,Topline
1,20171127,,,Top Start Date,Topline
2,20171130,,,Top End Date,Topline
3,3 Days,,,Top Duration,Topline
4,Email,,,Top Channel,Topline
5,"17,223",,,Top Incremental Dollars,Topline
6,148.88%,,,Top Incremental Percent,Topline
7,"28,791",,,Top Total Sales,Topline
8,6572,,,Top Target Population,Topline
9,91.32%,,,Top Target Percent,Topline


In [184]:
#Create independant vars for stacked bar out of dataframe
controlACV=pACV.iloc[0]['Control']
targetACV=pACV.iloc[0]['Target']
controlATV=pATV.iloc[0]['Control']
targetATV=pATV.iloc[0]['Target']
controlATF=pATF.iloc[0]['Control']
targetATF=pATF.iloc[0]['Target']

In [185]:
#Make a dir + dump all your resource files
%mkdir clcm-test

#JSON
dS.to_json("clcm-test/json-1.json", orient="records")
sS.to_json("clcm-test/json-2.json", orient="records")
cD.to_json("clcm-test/json-3.json", orient="records")

#Stacked bar data .txt
%store controlACV > clcm-test/controlACV.txt
%store targetACV > clcm-test/targetACV.txt
%store controlATV > clcm-test/controlATV.txt
%store targetATV > clcm-test/targetATV.txt
%store controlATF > clcm-test/controlATF.txt
%store targetATF > clcm-test/targetATF.txt

mkdir: clcm-test: File exists
Writing 'controlACV' (float64) to file 'clcm-test/controlACV.txt'.
Writing 'targetACV' (float64) to file 'clcm-test/targetACV.txt'.
Writing 'controlATV' (float64) to file 'clcm-test/controlATV.txt'.
Writing 'targetATV' (float64) to file 'clcm-test/targetATV.txt'.
Writing 'controlATF' (float64) to file 'clcm-test/controlATF.txt'.
Writing 'targetATF' (float64) to file 'clcm-test/targetATF.txt'.


In [186]:
#Downloads HTML files, concatnates JSON and adds stacked bar charts ↓

In [187]:
%%bash
mkdir -p clcm-test
cd clcm-test
curl "http://ux.rubikloud.com/assets/dev/clcm-test/clcm-1.html" -O 
curl "http://ux.rubikloud.com/assets/dev/clcm-test/clcm-2.html" -O 
curl "http://ux.rubikloud.com/assets/dev/clcm-test/clcm-3.html" -O 
curl "http://ux.rubikloud.com/assets/dev/clcm-test/clcm-4.html" -O 
cat clcm-1.html json-1.json clcm-2.html json-2.json clcm-3.html json-3.json clcm-4.html > -clcm-export.html

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  973k  100  973k    0     0   758k      0  0:00:01  0:00:01 --:--:--  816k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   936  100   936    0     0   4854      0 --:--:-- --:--:-- --:--:--  5005
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3455  100  3455    0     0  19988      0 --:--:-- --:--:-- --:--:-- 20813
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    32  100    32    0     0    129      0 --:--:-- --:--:-- --:--:--   131


In [188]:
%%bash
mkdir -p clcm-test
cd clcm-test

sed -i.bk-orig '/%%acv-target%%/{
    s/%%acv-target%%//g
    r targetACV.txt
}' -clcm-export.html

sed -i.bk '/%%acv-control%%/{
    s/%%acv-control%%//g
    r controlACV.txt
}' -clcm-export.html

sed -i.bk '/%%atv-target%%/{
    s/%%atv-target%%//g
    r targetATV.txt
}' -clcm-export.html

sed -i.bk '/%%atv-control%%/{
    s/%%atv-control%%//g
    r controlATV.txt
}' -clcm-export.html

sed -i.bk '/%%atf-target%%/{
    s/%%atf-target%%//g
    r targetATF.txt
}' -clcm-export.html

sed -i.bk '/%%atf-control%%/{
    s/%%atf-control%%//g
    r controlATF.txt
}' -clcm-export.html